In [2]:
!pip install geopandas folium shapely --quiet

import geopandas as gpd
from google.colab import files
import folium
from shapely.ops import unary_union
from shapely.geometry import MultiLineString
from IPython.display import display

# Step 3 — SafeLoop map builder (Colab)
# - Upload routes + DPS zone
# - Clip routes to DPS zone
# - Color-code options:
#     Red   = most efficient (lowest alpha)
#     Blue  = balanced (middle alpha)
#     Green = safest (highest alpha)

# 1) Upload files
print("Upload your SafeLoop routes GeoJSON (e.g., safeloops_3.0mi.geojson)...")
uploaded_routes = files.upload()
if not uploaded_routes:
    raise ValueError("No routes file uploaded.")
routes_path = list(uploaded_routes.keys())[0]
print("Using routes file:", routes_path)

print("\nUpload your USC DPS zone GeoJSON (usc_dps_upc_zone.geojson)...")
uploaded_dps = files.upload()
if not uploaded_dps:
    raise ValueError("No DPS file uploaded.")
dps_path = list(uploaded_dps.keys())[0]
print("Using DPS file:", dps_path)

# 2) Load data
loops_gdf = gpd.read_file(routes_path).to_crs(epsg=4326)
dps_gdf   = gpd.read_file(dps_path).to_crs(epsg=4326)

print("\nRoutes loaded:", len(loops_gdf))
print("DPS polygons loaded:", len(dps_gdf))

# 3) Clip routes to DPS zone
dps_geom = unary_union(dps_gdf.geometry)

loops_clipped = loops_gdf.copy()
loops_clipped["geometry"] = loops_clipped.geometry.intersection(dps_geom)
loops_clipped = loops_clipped[~loops_clipped.geometry.is_empty]

print(f"Clipped to DPS zone: {len(loops_gdf)} -> {len(loops_clipped)} routes")

# 4) Sort so colors always mean the same thing (by alpha)
#    lowest alpha = red (efficient), middle = blue, highest = green (safest)
if "alpha" not in loops_clipped.columns:
    raise ValueError("Your routes file is missing the 'alpha' column. Re-generate routes with alpha included.")

loops_sorted = loops_clipped.sort_values("alpha", ascending=True).reset_index(drop=True)

route_styles = [
    dict(label="Most efficient", line_color="#ff4b4b", marker_color="red"),
    dict(label="Balanced",       line_color="#4b7bff", marker_color="blue"),
    dict(label="Safest",         line_color="#4bff88", marker_color="green"),
]

# 5) Build Folium map
centroid = dps_geom.centroid
m = folium.Map(location=[centroid.y, centroid.x], zoom_start=14)

# DPS layer
folium.GeoJson(
    dps_gdf,
    name="USC DPS patrol zone",
    style_function=lambda x: {
        "fillColor": "#3388ff",
        "color": "#3388ff",
        "weight": 2,
        "fillOpacity": 0.15,
    },
    tooltip="USC DPS patrol zone",
).add_to(m)

# Add routes
for idx, row in loops_sorted.iterrows():
    style = route_styles[min(idx, 2)]  # expects 3 routes; clamps if fewer
    geom = row.geometry

    # For MultiLineString, pick the first part for start/end markers
    if isinstance(geom, MultiLineString):
        line = list(geom.geoms)[0]
    else:
        line = geom

    coords = list(line.coords)
    start_lon, start_lat = coords[0]
    end_lon, end_lat     = coords[-1]

    # Small offset so markers don't overlap
    offset = (idx - 1) * 0.00005
    start_lat_off = start_lat + offset
    end_lat_off   = end_lat + offset

    route_name = str(row.get("name", f"Route {idx+1}"))

    tooltip_html = (
        f"<b>{route_name}</b><br>"
        f"Option: {style['label']}<br>"
        f"{row.get('distance_miles', float('nan')):.2f} mi | {row.get('distance_km', float('nan')):.2f} km<br>"
        f"avg safety: {row.get('avg_safety', float('nan')):.1f}<br>"
        f"alpha: {row['alpha']:.2f}"
    )

    fg = folium.FeatureGroup(name=f"{style['label']} — {route_name}")

    folium.GeoJson(
        geom.__geo_interface__,
        style_function=lambda feat, c=style["line_color"]: {
            "color": c,
            "weight": 5,
            "opacity": 0.9,
        },
        tooltip=folium.Tooltip(tooltip_html, sticky=True),
    ).add_to(fg)

    folium.Marker(
        [start_lat_off, start_lon],
        icon=folium.Icon(color=style["marker_color"], icon="play", prefix="glyphicon"),
        tooltip=f"Start: {style['label']}",
    ).add_to(fg)

    folium.Marker(
        [end_lat_off, end_lon],
        icon=folium.Icon(color=style["marker_color"], icon="stop", prefix="glyphicon"),
        tooltip=f"End: {style['label']}",
    ).add_to(fg)

    fg.add_to(m)

folium.LayerControl(collapsed=False).add_to(m)

# Save + display
html_path = "safeloop_map.html"
m.save(html_path)
print("\nSaved HTML map:", html_path)

display(m)

from IPython.display import IFrame
IFrame("safeloop_map.html", width=1000, height=650)

Upload your SafeLoop routes GeoJSON (e.g., safeloops_3.0mi.geojson)...


Saving safeloops_4.0mi (1).geojson to safeloops_4.0mi (1).geojson
Using routes file: safeloops_4.0mi (1).geojson

Upload your USC DPS zone GeoJSON (usc_dps_upc_zone.geojson)...


Saving usc_dps_upc_zone (5).geojson to usc_dps_upc_zone (5) (1).geojson
Using DPS file: usc_dps_upc_zone (5) (1).geojson

Routes loaded: 3
DPS polygons loaded: 1
Clipped to DPS zone: 3 -> 3 routes

Saved HTML map: safeloop_map.html
